<a href="https://colab.research.google.com/github/Malanidhruv/Python-projects/blob/main/backtesting_Nifty_low%26high.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pya3
%pip install nsetools

In [ ]:
from pya3 import *
from datetime import datetime, timedelta, time
from nsetools import Nse
import holidays
from google.colab import drive
import pandas as pd

def get_user_credentials():
    """Prompt the user for their credentials and return them."""
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice(user_id, api_key):
    """Initialize the AliceBlue session."""
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()  # Log in to AliceBlue
    return alice

def get_market_holidays(year):
    """Return a list of market holidays for the given year."""
    market_holidays = holidays.India(years=year)
    # Add Mahashivratri as a market holiday on 08-03-2024
    market_holidays.append(datetime(2024, 11, 20).date())
    return market_holidays

def find_previous_trading_day(current_date, market_holidays):
    """Find the previous trading day, excluding weekends and holidays."""
    previous_trading_day = current_date - timedelta(days=1)
    while previous_trading_day.weekday() >= 5 or previous_trading_day in market_holidays:
        previous_trading_day -= timedelta(days=1)
    return previous_trading_day

def fetch_historical_data(alice, instrument, date):
    """Fetch historical data for the given instrument on the specified date."""
    from_datetime = datetime.combine(date, datetime.min.time())
    to_datetime = datetime.combine(date, datetime.max.time())
    interval = "D"
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    return historical_data

def next_thursday():
    """Find the next Thursday from the current date."""
    today = datetime.today().date()
    days_ahead = 3 - today.weekday()
    if days_ahead <= 0:  # Target day has already happened this week
        days_ahead += 7
    return today + timedelta(days=days_ahead)

def calculate_strike_prices(closing_price):
    """Calculate call and put option strike prices based on the closing price."""
    ce_strike_price = int(closing_price / 50) * 50
    pe_strike_price = ce_strike_price + 50
    return ce_strike_price, pe_strike_price

def construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price):
    """Construct option symbols for call and put options."""
    symbol_ce = f'NIFTY{expiry_date.strftime("%d%b%y")}C{ce_strike_price}'
    symbol_pe = f'NIFTY{expiry_date.strftime("%d%b%y")}P{pe_strike_price}'
    return symbol_ce, symbol_pe

def fetch_highest_high(alice, symbol, from_datetime, to_datetime, interval):
    """Fetch the highest high for the given symbol."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    highest_high = historical_data['high'].max()
    return round(float(highest_high))

def fetch_lowest_low(alice, symbol, from_datetime, to_datetime, interval):
    """Fetch the lowest low for the given symbol."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    lowest_low = historical_data['low'].min()  # Use min() to get the lowest low
    return round(float(lowest_low))

def place_order(alice, transaction_type, symbol, price_value, quantity=25):
    """Place an order for the given symbol."""
    if not isinstance(price_value, (float, int)):
        raise ValueError(f"Price must be a float or int, got {type(price_value).__name__}")

    price_value = float(price_value)  # Ensure price_value is a float

    return alice.place_order(
        transaction_type=transaction_type,
        instrument=alice.get_instrument_by_symbol('NFO', symbol),
        quantity=quantity,
        order_type=OrderType.StopLossLimit,
        product_type=ProductType.BracketOrder,
        price= price_value,
        trigger_price=price_value,
        stop_loss=15.0,
        square_off=50.0,
        trailing_sl=0.0,
        is_amo=True,
 order_tag='order1'
    )

def fetch_current_data(alice, symbol, from_time, to_time):
    """Fetch current data for the given symbol within the specified time range."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    current_data = alice.get_historical(instrument, from_time, to_time, interval='1')
    return current_data

def main():
    # Step 1: Mount Google Drive
    drive.mount('/content/drive')

    # Step 2: Load the CSV file
    nfo_file_path = '/content/drive/My Drive/NFO.csv'  # Change this path accordingly
    nfo_data = pd.read_csv(nfo_file_path)

    # Get user credentials and initialize AliceBlue
    user_id, api_key = get_user_credentials()
    alice = initialize_alice(user_id, api_key)

    # Create an instance of Nse
    nse = Nse()

    # Get market holidays for the current year
    current_date = datetime.now()
    market_holidays = get_market_holidays(current_date.year)

    # Find the previous trading day
    previous_trading_day = find_previous_trading_day(current_date, market_holidays)
    print(f"Previous Trading Day: {previous_trading_day}")

    # Fetch historical data
    instrument = alice.get_instrument_by_symbol('NFO', 'NIFTY26DEC24F')
    historical_data = fetch_historical_data(alice, instrument, previous_trading_day)

    last_close_price = historical_data['close'].iloc[-1]
    print("Closing Price of the Last Minute:", last_close_price)

    # Calculate strike prices
    ce_strike_price, pe_strike_price = calculate_strike_prices(last_close_price)

    # Get the next Thursday for expiry date
    expiry_date = next_thursday()

    # Construct option symbols
    symbol_ce, symbol_pe = construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price)
    print("CE Option Symbol:", symbol_ce)
    print("PE Option Symbol:", symbol_pe)

    # Define from_datetime, to_datetime, and interval for fetching highs and lows
    from_datetime = datetime.combine(previous_trading_day, datetime.min.time())
    to_datetime = datetime.combine(previous_trading_day, datetime.max.time())
    interval = "1"  # ["1", "D"]

    # Fetch and print current data for CE and PE from 9:15:29 to 15:17:29
    from_time = datetime.combine(current_date, time(9, 15, 29))
    to_time = datetime.combine(current_date, time(15, 17, 29))

    print("Fetching current data for CE and PE options...")
    ce_current_data = fetch_current_data(alice, symbol_ce, from_time, to_time)
    pe_current_data = fetch_current_data(alice, symbol_pe, from_time, to_time)

    print("Current Data for CE:")
    print(ce_current_data)

    print("Current Data for PE:")
    print(pe_current_data)

    # Get the highest high and place orders for CE
    highest_high_ce = fetch_highest_high(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Highest High CE Value:", highest_high_ce)
    print(place_order(alice, TransactionType.Sell, symbol_ce, highest_high_ce))

    # Get the highest high and place orders for PE
    highest_high_pe = fetch_highest_high(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Highest High PE Value:", highest_high_pe)
    print(place_order(alice, TransactionType.Sell, symbol_pe, highest_high_pe))

    # Get the lowest low and place orders for CE
    lowest_low_ce = fetch_lowest_low(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low CE Value:", lowest_low_ce)
    print(place_order(alice, TransactionType.Buy, symbol_ce, lowest_low_ce))

    # Get the lowest low and place orders for PE
    lowest_low_pe = fetch_lowest_low(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low PE Value:", lowest_low_pe)
    print(place_order(alice, TransactionType.Buy, symbol_pe, lowest_low_pe))

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter your user ID: 1141826
Enter your API key: nGhoLNREiDSHvBgjyGGQhltRljzoQX9XqvbLoAXoP6brZivMUymtMumvpK5bvqlkXyGcqVXbtmfanyXqixyRSo3DVSLdusxtTfYZIi9n0afG4ixXUOg0jS2uVOKm6kWY
Previous Trading Day: 2024-11-28 07:36:16.131835
Closing Price of the Last Minute: 24115.25
CE Option Symbol: NIFTY05Dec24C24100
PE Option Symbol: NIFTY05Dec24P24150
Fetching current data for CE and PE options...
Current Data for CE:
                datetime    open    high     low   close    volume
0    2024-11-29 09:15:59  148.05  161.75  141.00  148.45  629500.0
1    2024-11-29 09:16:59  147.80  150.85  146.10  150.25  401450.0
2    2024-11-29 09:17:59  150.45  154.50  148.00  154.00  287825.0
3    2024-11-29 09:18:59  153.60  155.35  152.05  153.10  227450.0
4    2024-11-29 09:19:59  153.20  153.20  148.00  149.10  213275.0
..                   ...     ...     ...     ...     ...  

# Experimenting backtesting codes

In [ ]:
from pya3 import *
from datetime import datetime, timedelta, time
from nsetools import Nse
import holidays
from google.colab import drive
import pandas as pd

def get_user_credentials():
    """Prompt the user for their credentials and return them."""
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

def initialize_alice(user_id, api_key):
    """Initialize the AliceBlue session."""
    alice = Aliceblue(user_id=user_id, api_key=api_key)
    alice.get_session_id()  # Log in to AliceBlue
    return alice

def get_market_holidays(year):
    """Return a list of market holidays for the given year."""
    market_holidays = holidays.India(years=year)
    # Add Mahashivratri as a market holiday on 08-03-2024
    market_holidays.append(datetime(2024, 11, 20).date())
    return market_holidays

def find_previous_trading_day(current_date, market_holidays):
    """Find the previous trading day, excluding weekends and holidays."""
    previous_trading_day = current_date - timedelta(days=1)
    while previous_trading_day.weekday() >= 5 or previous_trading_day in market_holidays:
        previous_trading_day -= timedelta(days=1)
    return previous_trading_day

def fetch_historical_data(alice, instrument, date):
    """Fetch historical data for the given instrument on the specified date."""
    from_datetime = datetime.combine(date, datetime.min.time())
    to_datetime = datetime.combine(date, datetime.max.time())
    interval = "D"
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    return historical_data

def next_thursday():
    """Find the next Thursday from the current date."""
    today = datetime.today().date()
    days_ahead = 3 - today.weekday()
    if days_ahead <= 0:  # Target day has already happened this week
        days_ahead += 7
    return today + timedelta(days=days_ahead)

def calculate_strike_prices(closing_price):
    """Calculate call and put option strike prices based on the closing price."""
    ce_strike_price = int(closing_price / 50) * 50
    pe_strike_price = ce_strike_price + 50
    return ce_strike_price, pe_strike_price

def construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price):
    """Construct option symbols for call and put options."""
    symbol_ce = f'NIFTY{expiry_date.strftime("%d%b%y")}C{ce_strike_price}'
    symbol_pe = f'NIFTY{expiry_date.strftime("%d%b%y")}P{pe_strike_price}'
    return symbol_ce, symbol_pe

def fetch_highest_high(alice, symbol, from_datetime, to_datetime, interval):
    """Fetch the highest high for the given symbol."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    highest_high = historical_data['high'].max()
    return round(float(highest_high))

def fetch_lowest_low(alice, symbol, from_datetime, to_datetime, interval):
    """Fetch the lowest low for the given symbol."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
    lowest_low = historical_data['low'].min()  # Use min() to get the lowest low
    return round(float(lowest_low))

def place_order(alice, transaction_type, symbol, price_value, quantity=25):
    """Place an order for the given symbol."""
    if not isinstance(price_value, (float, int)):
        raise ValueError(f"Price must be a float or int, got {type(price_value).__name__}")

    price_value = float(price_value)  # Ensure price_value is a float

    return alice.place_order(
        transaction_type=transaction_type,
        instrument=alice.get_instrument_by_symbol('NFO', symbol),
        quantity=quantity,
        order_type=OrderType.StopLossLimit,
        product_type=ProductType.BracketOrder,
        price= price_value,
        trigger_price=price_value,
        stop_loss=15.0,
        square_off=50.0,
        trailing_sl=0.0,
        is_amo=True,
 order_tag='order1'
    )

def fetch_current_data(alice, symbol, from_time, to_time):
    """Fetch current data for the given symbol within the specified time range."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    current_data = alice.get_historical(instrument, from_time, to_time, interval='1')
    return current_data

def main():
    # Step 1: Mount Google Drive
    drive.mount('/content/drive')

    # Step 2: Load the CSV file
    nfo_file_path = '/content/drive/My Drive/NFO.csv'  # Change this path accordingly
    nfo_data = pd.read_csv(nfo_file_path)

    # Get user credentials and initialize AliceBlue
    user_id, api_key = get_user_credentials()
    alice = initialize_alice(user_id, api_key)

    # Create an instance of Nse
    nse = Nse()

    # Get market holidays for the current year
    current_date = datetime.now()
    market_holidays = get_market_holidays(current_date.year)

    # Find the previous trading day
    previous_trading_day = find_previous_trading_day(current_date, market_holidays)
    print(f"Previous Trading Day: {previous_trading_day}")

    # Fetch historical data
    instrument = alice.get_instrument_by_symbol('NFO', 'NIFTY26DEC24F')
    historical_data = fetch_historical_data(alice, instrument, previous_trading_day)

    last_close_price = historical_data['close'].iloc[-1]
    print("Closing Price of the Last Minute:", last_close_price)

    # Calculate strike prices
    ce_strike_price, pe_strike_price = calculate_strike_prices(last_close_price)

    # Get the next Thursday for expiry date
    expiry_date = next_thursday()

    # Construct option symbols
    symbol_ce, symbol_pe = construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price)
    print("CE Option Symbol:", symbol_ce)
    print("PE Option Symbol:", symbol_pe)

    # Define from_datetime, to_datetime, and interval for fetching highs and lows
    from_datetime = datetime.combine(previous_trading_day, datetime.min.time())
    to_datetime = datetime.combine(previous_trading_day, datetime.max.time())
    interval = "1"  # ["1", "D"]

    # Fetch and print current data for CE and PE from 9:15:29 to 15:17:29
    from_time = datetime.combine(current_date, time(9, 15, 29))
    to_time = datetime.combine(current_date, time(15, 17, 29))

    print("Fetching current data for CE and PE options...")
    ce_current_data = fetch_current_data(alice, symbol_ce, from_time, to_time)
    pe_current_data = fetch_current_data(alice, symbol_pe, from_time, to_time)

    print("Current Data for CE:")
    print(ce_current_data)

    print("Current Data for PE:")
    print(pe_current_data)



    # Get the highest high and place orders for CE
    highest_high_ce = fetch_highest_high(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Highest High CE Value:", highest_high_ce)
    print(place_order(alice, TransactionType.Sell, symbol_ce, highest_high_ce))

    # Get the highest high and place orders for PE
    highest_high_pe = fetch_highest_high(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Highest High PE Value:", highest_high_pe)
    print(place_order(alice, TransactionType.Sell, symbol_pe, highest_high_pe))

    # Get the lowest low and place orders for CE
    lowest_low_ce = fetch_lowest_low(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low CE Value:", lowest_low_ce)
    print(place_order(alice, TransactionType.Buy, symbol_ce, lowest_low_ce))

    # Get the lowest low and place orders for PE
    lowest_low_pe = fetch_lowest_low(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low PE Value:", lowest_low_pe)
    print(place_order(alice, TransactionType.Buy, symbol_pe, lowest_low_pe))

if __name__ == "__main__":
    main()

In [ ]:
from pya3 import *
from datetime import datetime, timedelta, time
from nsetools import Nse
import holidays
from google.colab import drive
import pandas as pd

def next_thursday():
    """Find the next Thursday from the current date."""
    today = datetime.today().date()
    days_ahead = 3 - today.weekday()
    if days_ahead <= 0:  # Target day has already happened this week
        days_ahead += 7
    return today + timedelta(days=days_ahead)

def fetch_current_data(alice, symbol, from_time, to_time):
    """Fetch current data for the given symbol within the specified time range."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    current_data = alice.get_historical(instrument, from_time, to_time, interval='1')
    return current_data

# Include the backtest function here as defined earlier
def backtest(current_data, lowest_low, target_points=50, stop_loss_points=10):
    # Set the target and stop-loss levels
    target_price = lowest_low - target_points
    stop_loss_price = lowest_low + stop_loss_points

    # Initialize variables to track trade status
    trade_open = False
    entry_time = None
    exit_time = None
    result = None  # 'Win' or 'Loss'

    # Iterate through the current data
    for index, row in current_data.iterrows():
        timestamp = row['datetime']
        high = row['high']
        low = row['low']


        if not trade_open:
            # Check if we should enter the trade (if the buying price is touched)
            if low <= lowest_low <= high:
                # Buy order is triggered
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened at {entry_time} with entry price {lowest_low}")

        if trade_open:
            # Check if stop-loss or target is hit
            if low <= stop_loss_price:
                # Stop-loss is hit
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit at {exit_time} with price {stop_loss_price}")
                break
            elif high >= target_price:
                # Target is hit
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit at {exit_time} with price {target_price}")
                break

    # If trade is still open by the end of the data, log the result
    if trade_open and result is None:
        result = 'Open'  # Trade is still open

    return {
        'Entry Time': entry_time,
        'Exit Time': exit_time,
        'Result': result
    }

# Other functions remain unchanged...

def main():
    # Step 1: Mount Google Drive
    drive.mount('/content/drive')

    # Step 2: Load the CSV file
    nfo_file_path = '/content/drive/My Drive/NFO.csv'  # Change this path accordingly
    nfo_data = pd.read_csv(nfo_file_path)

    # Get user credentials and initialize AliceBlue
    user_id, api_key = get_user_credentials()
    alice = initialize_alice(user_id, api_key)

    # Create an instance of Nse
    nse = Nse()

    # Get market holidays for the current year
    current_date = datetime.now()
    market_holidays = get_market_holidays(current_date.year)

    # Find the previous trading day
    previous_trading_day = find_previous_trading_day(current_date, market_holidays)
    print(f"Previous Trading Day: {previous_trading_day}")

    # Fetch historical data
    instrument = alice.get_instrument_by_symbol('NFO', 'NIFTY26DEC24F')
    historical_data = fetch_historical_data(alice, instrument, previous_trading_day)

    last_close_price = historical_data['close'].iloc[-1]
    print("Closing Price of the Last Minute:", last_close_price)

    # Calculate strike prices
    ce_strike_price, pe_strike_price = calculate_strike_prices(last_close_price)

    # Get the next Thursday for expiry date
    expiry_date = next_thursday()

    # Construct option symbols
    symbol_ce, symbol_pe = construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price)
    print("CE Option Symbol:", symbol_ce)
    print("PE Option Symbol:", symbol_pe)

    # Define from_datetime, to_datetime, and interval for fetching highs and lows
    from_datetime = datetime.combine(previous_trading_day, datetime.min.time())
    to_datetime = datetime.combine(previous_trading_day, datetime.max.time())
    interval = "1"  # ["1", "D"]

    # Fetch and print current data for CE and PE from 9:15:29 to 15:17:29
    from_time = datetime.combine(current_date, time(9, 15, 29))
    to_time = datetime.combine(current_date, time(15, 17, 29))

    print("Fetching current data for CE and PE options...")
    ce_current_data = fetch_current_data(alice, symbol_ce, from_time, to_time)
    pe_current_data = fetch_current_data(alice, symbol_pe, from_time, to_time)

    print("Current Data for CE:")
    print(ce_current_data)

    print("Current Data for PE:")
    print(pe_current_data)

    # Get the highest high and place orders for CE
    highest_high_ce = fetch_highest_high(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Highest High CE Value:", highest_high_ce)
    print(place_order(alice, TransactionType.Sell, symbol_ce, highest_high_ce))

    # Get the highest high and place orders for PE
    highest_high_pe = fetch_highest_high(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Highest High PE Value:", highest_high_pe)
    print(place_order(alice, TransactionType.Sell, symbol_pe, highest_high_pe))

    # Get the lowest low and place orders for CE
    lowest_low_ce = fetch_lowest_low(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low CE Value:", lowest_low_ce)
    print(place_order(alice, TransactionType.Buy, symbol_ce, lowest_low_ce))

    # Get the lowest low and place orders for PE
    lowest_low_pe = fetch_lowest_low(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low PE Value:", lowest_low_pe)
    print(place_order(alice, TransactionType.Buy, symbol_pe, lowest_low_pe))

    # Perform backtesting for CE and PE
    backtest_ce_result = backtest(ce_current_data, lowest_low_ce)
    backtest_pe_result = backtest(pe_current_data, lowest_low_pe)

    # Output the backtest results
    print("CE Backtest Result:", backtest_ce_result)
    print("PE Backtest Result:", backtest_pe_result)

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter your user ID: 1141826
Enter your API key: nGhoLNREiDSHvBgjyGGQhltRljzoQX9XqvbLoAXoP6brZivMUymtMumvpK5bvqlkXyGcqVXbtmfanyXqixyRSo3DVSLdusxtTfYZIi9n0afG4ixXUOg0jS2uVOKm6kWY
Previous Trading Day: 2024-11-28 18:02:37.366753
Closing Price of the Last Minute: 24115.25
CE Option Symbol: NIFTY05Dec24C24100
PE Option Symbol: NIFTY05Dec24P24150
Fetching current data for CE and PE options...
Current Data for CE:
                datetime    open    high     low   close    volume
0    2024-11-29 09:15:59  148.05  161.75  141.00  148.45  629500.0
1    2024-11-29 09:16:59  147.80  150.85  146.10  150.25  401450.0
2    2024-11-29 09:17:59  150.45  154.50  148.00  154.00  287825.0
3    2024-11-29 09:18:59  153.60  155.35  152.05  153.10  227450.0
4    2024-11-29 09:19:59  153.20  153.20  148.00  149.10  213275.0
..                   ...     ...     ...     ...     ...  

# Low Backtesting

In [ ]:
from pya3 import *
from datetime import datetime, timedelta, time
from nsetools import Nse
import holidays
from google.colab import drive
import pandas as pd

def next_thursday():
    """Find the next Thursday from the current date."""
    today = datetime.today().date()
    days_ahead = 3 - today.weekday()
    if days_ahead <= 0:  # Target day has already happened this week
        days_ahead += 7
    return today + timedelta(days=days_ahead)

def fetch_current_data(alice, symbol, from_time, to_time):
    """Fetch current data for the given symbol within the specified time range."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    current_data = alice.get_historical(instrument, from_time, to_time, interval='1')
    return current_data


def backtest(current_data, trigger_price, trade_type='Buy', target_points=50, stop_loss_points=15):
    """
    Backtesting function for CE and PE trades.
    :param current_data: DataFrame with current data for CE or PE.
    :param trigger_price: Entry price for the trade.
    :param trade_type: 'Buy' or 'Sell'.
    :param target_points: Profit target in points.
    :param stop_loss_points: Stop-loss in points.
    :return: Dictionary with backtest results.
    """
    if trade_type == 'Buy':
        target_price = trigger_price + target_points
        stop_loss_price = trigger_price - stop_loss_points
    elif trade_type == 'Sell':
        target_price = trigger_price - target_points
        stop_loss_price = trigger_price + stop_loss_points
    else:
        raise ValueError("Invalid trade type. Use 'Buy' or 'Sell'.")

    trade_open = False
    entry_time, exit_time, result = None, None, None

    for index, row in current_data.iterrows():
        timestamp = row['datetime']
        high = row['high']
        low = row['low']

        if not trade_open:
            if trade_type == 'Buy' and low <= trigger_price <= high:
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened (Buy) at {entry_time} with entry price {trigger_price}")
            elif trade_type == 'Sell' and low <= trigger_price <= high:
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened (Sell) at {entry_time} with entry price {trigger_price}")

        if trade_open:
            if trade_type == 'Buy' and low <= stop_loss_price:
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit (Buy) at {exit_time} with price {stop_loss_price}")
                break
            elif trade_type == 'Sell' and high >= stop_loss_price:
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit (Sell) at {exit_time} with price {stop_loss_price}")
                break
            elif trade_type == 'Buy' and high >= target_price:
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit (Buy) at {exit_time} with price {target_price}")
                break
            elif trade_type == 'Sell' and low <= target_price:
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit (Sell) at {exit_time} with price {target_price}")
                break

    if trade_open and result is None:
        result = 'Open'

    return {
        'Entry Time': entry_time,
        'Exit Time': exit_time,
        'Result': result,
        'Trade Type': trade_type
    }


# Other functions remain unchanged...

def main():
    # Step 1: Mount Google Drive
    drive.mount('/content/drive')

    # Step 2: Load the CSV file
    nfo_file_path = '/content/drive/My Drive/NFO.csv'  # Change this path accordingly
    nfo_data = pd.read_csv(nfo_file_path)

    # Get user credentials and initialize AliceBlue
    user_id, api_key = get_user_credentials()
    alice = initialize_alice(user_id, api_key)

    # Create an instance of Nse
    nse = Nse()

    # Get market holidays for the current year
    current_date = datetime.now()
    market_holidays = get_market_holidays(current_date.year)

    # Find the previous trading day
    previous_trading_day = find_previous_trading_day(current_date, market_holidays)
    print(f"Previous Trading Day: {previous_trading_day}")

    # Fetch historical data
    instrument = alice.get_instrument_by_symbol('NFO', 'NIFTY26DEC24F')
    historical_data = fetch_historical_data(alice, instrument, previous_trading_day)

    last_close_price = historical_data['close'].iloc[-1]
    print("Closing Price of the Last Minute:", last_close_price)

    # Calculate strike prices
    ce_strike_price, pe_strike_price = calculate_strike_prices(last_close_price)

    # Get the next Thursday for expiry date
    expiry_date = next_thursday()

    # Construct option symbols
    symbol_ce, symbol_pe = construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price)
    print("CE Option Symbol:", symbol_ce)
    print("PE Option Symbol:", symbol_pe)

    # Define from_datetime, to_datetime, and interval for fetching highs and lows
    from_datetime = datetime.combine(previous_trading_day, datetime.min.time())
    to_datetime = datetime.combine(previous_trading_day, datetime.max.time())
    interval = "1"  # ["1", "D"]

    # Fetch and print current data for CE and PE from 9:15:29 to 15:17:29
    from_time = datetime.combine(current_date, time(9, 15, 29))
    to_time = datetime.combine(current_date, time(15, 17, 29))

    print("Fetching current data for CE and PE options...")
    ce_current_data = fetch_current_data(alice, symbol_ce, from_time, to_time)
    pe_current_data = fetch_current_data(alice, symbol_pe, from_time, to_time)


    # Get the highest high and place orders for CE
    highest_high_ce = fetch_highest_high(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Highest High CE Value:", highest_high_ce)
    print(place_order(alice, TransactionType.Sell, symbol_ce, highest_high_ce))

    # Get the highest high and place orders for PE
    highest_high_pe = fetch_highest_high(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Highest High PE Value:", highest_high_pe)
    print(place_order(alice, TransactionType.Sell, symbol_pe, highest_high_pe))

    # Get the lowest low and place orders for CE
    lowest_low_ce = fetch_lowest_low(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low CE Value:", lowest_low_ce)
    print(place_order(alice, TransactionType.Buy, symbol_ce, lowest_low_ce))

    # Get the lowest low and place orders for PE
    lowest_low_pe = fetch_lowest_low(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low PE Value:", lowest_low_pe)
    print(place_order(alice, TransactionType.Buy, symbol_pe, lowest_low_pe))

    print("Backtesting CE (Buy at low price)...")
    ce_buy_result = backtest(ce_current_data, lowest_low_ce, trade_type='Buy')

    print("Backtesting CE (Sell at high price)...")
    ce_sell_result = backtest(ce_current_data, highest_high_ce, trade_type='Sell')

    print("Backtesting PE (Buy at low price)...")
    pe_buy_result = backtest(pe_current_data, lowest_low_pe, trade_type='Buy')

    print("Backtesting PE (Sell at high price)...")
    pe_sell_result = backtest(pe_current_data, highest_high_pe, trade_type='Sell')

    # Print backtest results
    print("CE Buy Backtest Result:", ce_buy_result)
    print("CE Sell Backtest Result:", ce_sell_result)
    print("PE Buy Backtest Result:", pe_buy_result)
    print("PE Sell Backtest Result:", pe_sell_result)

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter your user ID: 1141826
Enter your API key: nGhoLNREiDSHvBgjyGGQhltRljzoQX9XqvbLoAXoP6brZivMUymtMumvpK5bvqlkXyGcqVXbtmfanyXqixyRSo3DVSLdusxtTfYZIi9n0afG4ixXUOg0jS2uVOKm6kWY
Previous Trading Day: 2024-11-28 18:00:39.178773
Closing Price of the Last Minute: 24115.25
CE Option Symbol: NIFTY05Dec24C24100
PE Option Symbol: NIFTY05Dec24P24150
Fetching current data for CE and PE options...
Rounded Highest High CE Value: 375
{'stat': 'Ok', 'NOrdNo': '24112900254881'}
Rounded Highest High PE Value: 311
{'stat': 'Ok', 'NOrdNo': '24112900254882'}
Rounded Lowest Low CE Value: 126
{'stat': 'Ok', 'NOrdNo': '24112900254883'}
Rounded Lowest Low PE Value: 77
{'stat': 'Ok', 'NOrdNo': '24112900254884'}
Backtesting CE (Buy at low price)...
Backtesting CE (Sell at high price)...
Backtesting PE (Buy at low price)...
Backtesting PE (Sell at high price)...
CE Buy Backtest Result

# High Volatility

In [ ]:
from pya3 import *
from datetime import datetime, timedelta, time
from nsetools import Nse
import holidays
from google.colab import drive
import pandas as pd

def next_thursday():
    """Find the next Thursday from the current date."""
    today = datetime.today().date()
    days_ahead = 3 - today.weekday()
    if days_ahead <= 0:  # Target day has already happened this week
        days_ahead += 7
    return today + timedelta(days=days_ahead)

def fetch_current_data(alice, symbol, from_time, to_time):
    """Fetch current data for the given symbol within the specified time range."""
    instrument = alice.get_instrument_by_symbol('NFO', symbol)
    current_data = alice.get_historical(instrument, from_time, to_time, interval='1')
    return current_data

def backtest(current_data, trigger_price, trade_type='Buy', target_points=50, stop_loss_points=15):
    """
    Backtesting function for CE and PE trades.
    :param current_data: DataFrame with current data for CE or PE.
    :param trigger_price: Entry price for the trade.
    :param trade_type: 'Buy' or 'Sell'.
    :param target_points: Profit target in points.
    :param stop_loss_points: Stop-loss in points.
    :return: Dictionary with backtest results.
    """
    if trade_type == 'Buy':
        target_price = trigger_price + target_points
        stop_loss_price = trigger_price - stop_loss_points
    elif trade_type == 'Sell':
        target_price = trigger_price - target_points
        stop_loss_price = trigger_price + stop_loss_points
    else:
        raise ValueError("Invalid trade type. Use 'Buy' or 'Sell'.")

    trade_open = False
    entry_time, exit_time, result = None, None, None

    for index, row in current_data.iterrows():
        timestamp = row['datetime']
        high = row['high']
        low = row['low']

        if not trade_open:
            if trade_type == 'Buy' and low <= trigger_price <= high:
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened (Buy) at {entry_time} with entry price {trigger_price}")
            elif trade_type == 'Sell' and low <= trigger_price <= high:
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened (Sell) at {entry_time} with entry price {trigger_price}")

        if trade_open:
            if trade_type == 'Buy' and low <= stop_loss_price:
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit (Buy) at {exit_time} with price {stop_loss_price}")
                break
            elif trade_type == 'Sell' and high >= stop_loss_price:
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit (Sell) at {exit_time} with price {stop_loss_price}")
                break
            elif trade_type == 'Buy' and high >= target_price:
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit (Buy) at {exit_time} with price {target_price}")
                break
            elif trade_type == 'Sell' and low <= target_price:
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit (Sell) at {exit_time} with price {target_price}")
                break

    if trade_open and result is None:
        result = 'Open'

    return {
        'Entry Time': entry_time,
        'Exit Time': exit_time,
        'Result': result,
        'Trade Type': trade_type
    }


# Other functions remain unchanged...

def main():
    # Step 1: Mount Google Drive
    drive.mount('/content/drive')

    # Step 2: Load the CSV file
    nfo_file_path = '/content/drive/My Drive/NFO.csv'  # Change this path accordingly
    nfo_data = pd.read_csv(nfo_file_path)

    # Get user credentials and initialize AliceBlue
    user_id, api_key = get_user_credentials()
    alice = initialize_alice(user_id, api_key)

    # Create an instance of Nse
    nse = Nse()

    # Get market holidays for the current year
    current_date = datetime.now()
    market_holidays = get_market_holidays(current_date.year)

    # Find the previous trading day
    previous_trading_day = find_previous_trading_day(current_date, market_holidays)
    print(f"Previous Trading Day: {previous_trading_day}")

    # Fetch historical data
    instrument = alice.get_instrument_by_symbol('NFO', 'NIFTY26DEC24F')
    historical_data = fetch_historical_data(alice, instrument, previous_trading_day)

    last_close_price = historical_data['close'].iloc[-1]
    print("Closing Price of the Last Minute:", last_close_price)

    # Calculate strike prices
    ce_strike_price, pe_strike_price = calculate_strike_prices(last_close_price)

    # Get the next Thursday for expiry date
    expiry_date = next_thursday()

    # Construct option symbols
    symbol_ce, symbol_pe = construct_option_symbols(expiry_date, ce_strike_price, pe_strike_price)
    print("CE Option Symbol:", symbol_ce)
    print("PE Option Symbol:", symbol_pe)

    # Define from_datetime, to_datetime, and interval for fetching highs and lows
    from_datetime = datetime.combine(previous_trading_day, datetime.min.time())
    to_datetime = datetime.combine(previous_trading_day, datetime.max.time())
    interval = "1"  # ["1", "D"]

    # Fetch and print current data for CE and PE from 9:15:29 to 15:17:29
    from_time = datetime.combine(current_date, time(9, 15, 29))
    to_time = datetime.combine(current_date, time(15, 17, 29))

    print("Fetching current data for CE and PE options...")
    ce_current_data = fetch_current_data(alice, symbol_ce, from_time, to_time)
    pe_current_data = fetch_current_data(alice, symbol_pe, from_time, to_time)


    # Get the highest high and place orders for CE
    highest_high_ce = fetch_highest_high(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Highest High CE Value:", highest_high_ce)
    print(place_order(alice, TransactionType.Buy, symbol_ce, highest_high_ce))

    # Get the highest high and place orders for PE
    highest_high_pe = fetch_highest_high(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Highest High PE Value:", highest_high_pe)
    print(place_order(alice, TransactionType.Buy, symbol_pe, highest_high_pe))

    # Get the lowest low and place orders for CE
    lowest_low_ce = fetch_lowest_low(alice, symbol_ce, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low CE Value:", lowest_low_ce)
    print(place_order(alice, TransactionType.Sell, symbol_ce, lowest_low_ce))

    # Get the lowest low and place orders for PE
    lowest_low_pe = fetch_lowest_low(alice, symbol_pe, from_datetime, to_datetime, interval)
    print("Rounded Lowest Low PE Value:", lowest_low_pe)
    print(place_order(alice, TransactionType.Sell, symbol_pe, lowest_low_pe))

    print("Backtesting CE (Buy at low price)...")
    ce_buy_result = backtest(ce_current_data, lowest_low_ce, trade_type='Sell')

    print("Backtesting CE (Sell at high price)...")
    ce_sell_result = backtest(ce_current_data, highest_high_ce, trade_type='Buy')

    print("Backtesting PE (Buy at low price)...")
    pe_buy_result = backtest(pe_current_data, lowest_low_pe, trade_type='Sell')

    print("Backtesting PE (Sell at high price)...")
    pe_sell_result = backtest(pe_current_data, highest_high_pe, trade_type='Buy')

    # Print backtest results
    print("CE Buy Backtest Result:", ce_buy_result)
    print("CE Sell Backtest Result:", ce_sell_result)
    print("PE Buy Backtest Result:", pe_buy_result)
    print("PE Sell Backtest Result:", pe_sell_result)

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter your user ID: 1141826
Enter your API key: nGhoLNREiDSHvBgjyGGQhltRljzoQX9XqvbLoAXoP6brZivMUymtMumvpK5bvqlkXyGcqVXbtmfanyXqixyRSo3DVSLdusxtTfYZIi9n0afG4ixXUOg0jS2uVOKm6kWY
Previous Trading Day: 2024-11-28 17:59:55.897122
Closing Price of the Last Minute: 24115.25
CE Option Symbol: NIFTY05Dec24C24100
PE Option Symbol: NIFTY05Dec24P24150
Fetching current data for CE and PE options...
Rounded Highest High CE Value: 375
{'stat': 'Ok', 'NOrdNo': '24112900254867'}
Rounded Highest High PE Value: 311
{'stat': 'Ok', 'NOrdNo': '24112900254868'}
Rounded Lowest Low CE Value: 126
{'stat': 'Ok', 'NOrdNo': '24112900254870'}
Rounded Lowest Low PE Value: 77
{'stat': 'Ok', 'NOrdNo': '24112900254871'}
Backtesting CE (Buy at low price)...
Backtesting CE (Sell at high price)...
Backtesting PE (Buy at low price)...
Backtesting PE (Sell at high price)...
CE Buy Backtest Result

In [ ]:
def backtest(current_data, trigger_price, trade_type='Buy', target_points=50, stop_loss_points=10):
    """
    Backtesting function for CE and PE trades.
    :param current_data: DataFrame with current data for CE or PE.
    :param trigger_price: Entry price for the trade.
    :param trade_type: 'Buy' or 'Sell'.
    :param target_points: Profit target in points.
    :param stop_loss_points: Stop-loss in points.
    :return: Dictionary with backtest results.
    """
    if trade_type == 'Buy':
        target_price = trigger_price + target_points
        stop_loss_price = trigger_price - stop_loss_points
    elif trade_type == 'Sell':
        target_price = trigger_price - target_points
        stop_loss_price = trigger_price + stop_loss_points
    else:
        raise ValueError("Invalid trade type. Use 'Buy' or 'Sell'.")

    trade_open = False
    entry_time, exit_time, result = None, None, None

    for index, row in current_data.iterrows():
        timestamp = row['datetime']
        high = row['high']
        low = row['low']

        if not trade_open:
            if trade_type == 'Buy' and low <= trigger_price <= high:
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened (Buy) at {entry_time} with entry price {trigger_price}")
            elif trade_type == 'Sell' and low <= trigger_price <= high:
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened (Sell) at {entry_time} with entry price {trigger_price}")

        if trade_open:
            if trade_type == 'Buy' and low <= stop_loss_price:
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit (Buy) at {exit_time} with price {stop_loss_price}")
                break
            elif trade_type == 'Sell' and high >= stop_loss_price:
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit (Sell) at {exit_time} with price {stop_loss_price}")
                break
            elif trade_type == 'Buy' and high >= target_price:
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit (Buy) at {exit_time} with price {target_price}")
                break
            elif trade_type == 'Sell' and low <= target_price:
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit (Sell) at {exit_time} with price {target_price}")
                break

    if trade_open and result is None:
        result = 'Open'

    return {
        'Entry Time': entry_time,
        'Exit Time': exit_time,
        'Result': result,
        'Trade Type': trade_type
    }

    print("Backtesting CE (Buy at low price)...")
    ce_buy_result = backtest(ce_current_data, lowest_low_ce, trade_type='Buy')

    print("Backtesting CE (Sell at high price)...")
    ce_sell_result = backtest(ce_current_data, highest_high_ce, trade_type='Sell')

    print("Backtesting PE (Buy at low price)...")
    pe_buy_result = backtest(pe_current_data, lowest_low_pe, trade_type='Buy')

    print("Backtesting PE (Sell at high price)...")
    pe_sell_result = backtest(pe_current_data, highest_high_pe, trade_type='Sell')

    # Print backtest results
    print("CE Buy Backtest Result:", ce_buy_result)
    print("CE Sell Backtest Result:", ce_sell_result)
    print("PE Buy Backtest Result:", pe_buy_result)
    print("PE Sell Backtest Result:", pe_sell_result)

In [ ]:
#NIFTY
from pya3 import *


def get_user_credentials():
    user_id = input("Enter your user ID: ")
    api_key = input("Enter your API key: ")
    return user_id, api_key

# Example usage:
user_id, api_key = get_user_credentials()
alice = Aliceblue(user_id=user_id, api_key=api_key)
# Now you can use the 'alice' object in the rest of your code.



#you will get ok after successfully login in alice blue using the api
alice.get_session_id()

from datetime import datetime, timedelta
from nsetools import Nse
import holidays
import datetime



# Create an instance of Nse
nse = Nse()

# Get the list of market holidays for the year
market_holidays = holidays.India(years=datetime.datetime.now().year)

# Add Mahashivratri as a market holiday on 08-03-24
market_holidays.append(datetime.date(2024, 3, 8))

# Get the current date
current_date = datetime.datetime.now()

# Find the previous trading day
previous_trading_day = current_date - timedelta(days=1)
while previous_trading_day.weekday() >= 5 or previous_trading_day in market_holidays:
    previous_trading_day -= timedelta(days=1)


yesterday_date = previous_trading_day.date()

instrument = alice.get_instrument_by_symbol('NFO', 'NIFTY26DEC24F')
print(instrument)

from_datetime = datetime.datetime.combine(yesterday_date, datetime.time.min)  # From yesterday
to_datetime = datetime.datetime.combine(yesterday_date, datetime.time.max)  # To yesterday
interval = "D"  # ["1", "D"]
historical_data = alice.get_historical(instrument, from_datetime, to_datetime, interval)
print(historical_data)

last_close_price = historical_data['close'].iloc[-1]
print("Closing Price of the Last Minute:", last_close_price)

# Store the closing price for later use
closeprice = last_close_price

# Now you can use 'stored_close_price' in your subsequent code
print("Closing Price:", closeprice)

# Function to check if a given date is a holiday
def is_holiday(date):
    india_holidays = holidays.CountryHoliday('IND', prov='MH')
    return date in india_holidays

# Find the next Thursday from the current date
def next_thursday():
    today = datetime.date.today()
    days_ahead = 3 - today.weekday()  # 3 corresponds to Thursday
    if days_ahead <= 2:  # Target day has already happened this week
        days_ahead += 7
    return today + datetime.timedelta(days=days_ahead)

# Assuming you have the closing price stored in 'last_close_price'
closing_price = closeprice  # Replace this with the actual closing price from your data

# Calculate the nearest strike prices
ce_strike_price = int(closing_price / 50) * 50
pe_strike_price = ce_strike_price + 50

# Get the next Thursday for expiry date
expiry_date = next_thursday()

# Construct option symbols with expiry date and strike prices
symbol_ce = f'NIFTY{expiry_date.strftime("%d%b%y")}C{ce_strike_price}'
symbol_pe = f'NIFTY{expiry_date.strftime("%d%b%y")}P{pe_strike_price}'

print("CE Option Symbol:", symbol_ce)
print("PE Option Symbol:", symbol_pe)



# Define the instrument symbol_pe
instrument1 = alice.get_instrument_by_symbol('NFO', symbol_ce)

# Get historical data
historical_data = alice.get_historical(instrument1, from_datetime, to_datetime, interval)
print(historical_data)

# Assuming 'data' is a DataFrame
high_values = historical_data['low'].tolist()
print(high_values)

highest_high_ce = historical_data['low'].max()
print("Highest High Value:", highest_high_ce)

highest_high_ce = round(highest_high_ce)
highest_high_ce = float(highest_high_ce)
print("Rounded Highest High Value:", highest_high_ce)


# Define the instrument symbol_pe
instrument2 = alice.get_instrument_by_symbol('NFO', symbol_pe)


# Get historical data
historical_data = alice.get_historical(instrument2, from_datetime, to_datetime, interval)
print(historical_data)

# Assuming 'data' is a DataFrame
high_values = historical_data['low'].tolist()
print(high_values)

highest_high_pe = historical_data['low'].max()
print("Highest High Value:", highest_high_pe)

highest_high_pe = round(highest_high_pe)
highest_high_pe = float(highest_high_pe)
print("Rounded Highest High Value:", highest_high_pe)


#Placing order for ce
# Define a variable name as a string
CE_high = "highest_high_ce"

# Retrieve the variable value using globals()
price_value = globals()[CE_high]

print(
   alice.place_order(transaction_type = TransactionType.Buy,
                     instrument = alice.get_instrument_by_symbol('NFO', symbol_ce),
                     quantity = 25,
                     order_type = OrderType.StopLossLimit,
                     product_type = ProductType.BracketOrder,
                     price = price_value,  # Pass the retrieved value
                     trigger_price = price_value,
                     stop_loss =  100.0,
                     square_off = 10.0,
                     trailing_sl = 0.0,
                     is_amo = True,
                     order_tag='order1')
)



#placing order for pe
# Define a variable name as a string
PE_high = "highest_high_pe"

# Retrieve the variable value using globals()
price_value = globals()[PE_high]

print(
   alice.place_order(transaction_type = TransactionType.Buy,
                     instrument = alice.get_instrument_by_symbol('NFO', symbol_pe),
                     quantity = 25,
                     order_type = OrderType.StopLossLimit,
                     product_type = ProductType.BracketOrder,
                     price = price_value,  # Pass the retrieved value
                     trigger_price = price_value,
                     stop_loss =  100.0,
                     square_off = 10.0,
                     trailing_sl = 0.0,
                     is_amo = True,
                     order_tag='order1')
)


Enter your user ID: 1141826
Enter your API key: nGhoLNREiDSHvBgjyGGQhltRljzoQX9XqvbLoAXoP6brZivMUymtMumvpK5bvqlkXyGcqVXbtmfanyXqixyRSo3DVSLdusxtTfYZIi9n0afG4ixXUOg0jS2uVOKm6kWY
Instrument(exchange='NFO', token=35005, symbol='NIFTY', name='NIFTY26DEC24F', expiry='', lot_size=25)
              datetime      open     high       low     close    volume
0  2024-11-28 00:00:00  24460.25  24510.0  24056.05  24115.25  479877.0
Closing Price of the Last Minute: 24115.25
Closing Price: 24115.25
CE Option Symbol: NIFTY05Dec24C24100
PE Option Symbol: NIFTY05Dec24P24150
              datetime   open    high    low   close    volume
0  2024-11-28 00:00:00  340.2  374.95  126.0  140.95  870272.0
[126.0]
Highest High Value: 126.0
Rounded Highest High Value: 126.0
              datetime   open    high   low   close    volume
0  2024-11-28 00:00:00  100.8  310.55  77.2  265.15  242974.0
[77.2]
Highest High Value: 77.2
Rounded Highest High Value: 77.0
{'stat': 'Ok', 'NOrdNo': '24112900158289'}
{'stat': '

In [ ]:
from_datetime = datetime.datetime.combine(current_date, time(9, 15))
to_datetime = datetime.datetime.combine(current_date, time(15, 30))
interval = "1"  # ["1", "D"]

historical_data = alice.get_historical(instrument1, from_datetime, to_datetime, interval)
print(historical_data)


                datetime    open    high     low   close    volume
0    2024-11-29 09:15:59  148.05  161.75  141.00  148.45  629500.0
1    2024-11-29 09:16:59  147.80  150.85  146.10  150.25  401450.0
2    2024-11-29 09:17:59  150.45  154.50  148.00  154.00  287825.0
3    2024-11-29 09:18:59  153.60  155.35  152.05  153.10  227450.0
4    2024-11-29 09:19:59  153.20  153.20  148.00  149.10  213275.0
..                   ...     ...     ...     ...     ...       ...
235  2024-11-29 13:10:59  192.75  195.60  190.65  195.35  102975.0
236  2024-11-29 13:11:59  195.30  198.55  194.35  194.85  163550.0
237  2024-11-29 13:12:59  194.40  198.00  194.40  197.60  103975.0
238  2024-11-29 13:13:59  197.50  199.70  197.50  198.30  140425.0
239  2024-11-29 13:14:59  199.00  202.35  198.10  201.30  230650.0

[240 rows x 6 columns]


In [ ]:
from_datetime = datetime.datetime.combine(current_date, time(9, 15))
to_datetime = datetime.datetime.combine(current_date, time(15, 30))
interval = "1"  # ["1", "D"]

historical_data2 = alice.get_historical(instrument2, from_datetime, to_datetime, interval)
print(historical_data2)

                datetime    open    high     low   close    volume
0    2024-11-29 09:15:59  250.90  254.50  233.50  253.05   38900.0
1    2024-11-29 09:16:59  255.00  259.45  244.85  245.00   47550.0
2    2024-11-29 09:17:59  246.25  251.85  244.00  244.65   14950.0
3    2024-11-29 09:18:59  245.20  247.40  243.25  244.60   26100.0
4    2024-11-29 09:19:59  244.75  255.85  244.40  253.45   25725.0
..                   ...     ...     ...     ...     ...       ...
240  2024-11-29 13:15:59  162.50  163.45  160.35  162.55  142700.0
241  2024-11-29 13:16:59  162.10  164.35  161.50  162.95   73775.0
242  2024-11-29 13:17:59  162.90  164.50  161.55  162.70   58250.0
243  2024-11-29 13:18:59  162.85  162.85  158.90  159.45  113750.0
244  2024-11-29 13:19:59  159.25  159.40  152.80  153.70  184200.0

[245 rows x 6 columns]


In [ ]:
import pandas as pd

def backtest(historical_data, highest_high_ce, target_points=50, stop_loss_points=10):
    # Set the target and stop-loss levels
    target_price = highest_high_ce + target_points
    stop_loss_price = highest_high_ce - stop_loss_points

    # Initialize variables to track trade status
    trade_open = False
    entry_time = None
    exit_time = None
    result = None  # 'Win' or 'Loss'

    # Iterate through the historical data
    for index, row in historical_data.iterrows():
        timestamp = row['datetime']
        high = row['high']
        low = row['low']

        if not trade_open:
            # Check if we should enter the trade (if the buying price is touched)
            if low <= highest_high_ce <= high:
                # Buy order is triggered
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened at {entry_time} with entry price {highest_high_ce}")

        if trade_open:
            # Check if stop-loss or target is hit
            if low <= stop_loss_price:
                # Stop-loss is hit
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit at {exit_time} with price {stop_loss_price}")
                break
            elif high >= target_price:
                # Target is hit
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit at {exit_time} with price {target_price}")
                break

    # If trade is still open by the end of the day, log the result
    if trade_open and result is None:
        result = 'Open'  # Trade is still open

    return {
        'Entry Time': entry_time,
        'Exit Time': exit_time,
        'Result': result
    }

# Ensure 'datetime' column is in datetime format
historical_data['datetime'] = pd.to_datetime(historical_data['datetime'])

# Set your buying price
highest_high_ce

# Call the backtesting function
backtest_result = backtest(historical_data, highest_high_ce)

# Output the result
print(backtest_result)


{'Entry Time': None, 'Exit Time': None, 'Result': None}


In [ ]:
import pandas as pd

def backtest(historical_data2, highest_high_ce, target_points=50, stop_loss_points=10):
    # Set the target and stop-loss levels
    target_price = highest_high_ce + target_points
    stop_loss_price = highest_high_ce - stop_loss_points

    # Initialize variables to track trade status
    trade_open = False
    entry_time = None
    exit_time = None
    result = None  # 'Win' or 'Loss'

    # Iterate through the historical data
    for index, row in historical_data2.iterrows():
        timestamp = row['datetime']
        high = row['high']
        low = row['low']

        if not trade_open:
            # Check if we should enter the trade (if the buying price is touched)
            if low <= highest_high_ce <= high:
                # Buy order is triggered
                trade_open = True
                entry_time = timestamp
                print(f"Trade opened at {entry_time} with entry price {highest_high_ce}")

        if trade_open:
            # Check if stop-loss or target is hit
            if low <= stop_loss_price:
                # Stop-loss is hit
                exit_time = timestamp
                result = 'Loss'
                print(f"Stop-loss hit at {exit_time} with price {stop_loss_price}")
                break
            elif high >= target_price:
                # Target is hit
                exit_time = timestamp
                result = 'Win'
                print(f"Target hit at {exit_time} with price {target_price}")
                break

    # If trade is still open by the end of the day, log the result
    if trade_open and result is None:
        result = 'Open'  # Trade is still open

    return {
        'Entry Time': entry_time,
        'Exit Time': exit_time,
        'Result': result
    }

# Ensure 'datetime' column is in datetime format
historical_data2['datetime'] = pd.to_datetime(historical_data2['datetime'])

# Set your buying price
highest_high_ce

# Call the backtesting function
backtest_result = backtest(historical_data2, highest_high_ce)

# Output the result
print(backtest_result)


{'Entry Time': None, 'Exit Time': None, 'Result': None}
